In [ ]:
import os
import json
import yaml
import time
import shutil
import subprocess
import google.generativeai as genai
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List
from github import Github
from time import sleep
import logging

# Configure logging
LOG_FILE_PATH = './refactor_log.log'
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    handlers=[logging.FileHandler(LOG_FILE_PATH), logging.StreamHandler()])

REPO_URL = 'https://github.com/onepiece-iiith/RefactoringSampleRepo' 
# REPO_URL = 'https://github.com/onepiece-iiith/ScoreBoard-Project'
REPO_NAME = REPO_URL.strip().strip("/").split('/')[-1].replace('.git', '')
REPO_OWNER = REPO_URL.strip().strip("/").split('/')[-2]
WORKING_DIRECTORY_PATH = './WorkingDirectory'

REPO_LOCAL_PATH = os.path.join(WORKING_DIRECTORY_PATH, REPO_NAME)
REPO_CONTEXT_TREE_PATH = os.path.join("ContextTree", f"{REPO_NAME}.txt")

API_KEY = "KEY"
KNOWLEDGE_BASE_DESIGN_SMELLS_PATH = "KnowledgeBase/DesignSmell.yaml"

GITHUB_TOKEN = "KEY"
BASE_BRANCH = "main"
BRANCH_NAME = f"refactor-branch"
COMMIT_MESSAGE = "Refactored code to address design smells"
PR_TITLE = "Refactoring Updates"
PR_BODY = "This PR contains refactored code to address design smells."

MODEL = "gemini-1.5-flash"

class DesignSmell(BaseModel):
    Design_Smell: str = Field(description="The name of the design smell")
    # Description: str = Field(description="Description of the design smell")
    Files_Paths: List[str] = Field(description="List of file paths where the smell is found")
    Reasoning: str = Field(description="Reasoning or evidence supporting the presence of the smell")

class CodeBlock(BaseModel):
    File_Path: str = Field(description="File path where the smell is found")
    Updated_Code: str = Field(description="Entire Updated code for the file")

class CommitInfo(BaseModel):
    brach_name : str = Field(description="Branch name for the commit")
    pull_request_title: str = Field(description="Title for the pull request")
    pull_request_body: str = Field(description="Body for the pull request")
    commit_message: str = Field(description="Commit message for the changes") 

def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

def load_context(context_file_path):
    with open(context_file_path, 'r') as f:
        context = f.read()
    return context

def clone_repo(repo_url, repo_name, destination_path, patience=5):
    attempts = 0
    while attempts < patience:
        try:
            destination_path = os.path.join(destination_path, repo_name)
            
            # Check if the destination directory already exists
            if os.path.exists(destination_path):
                logging.warning(f"Destination path '{destination_path}' already exists. Removing it...")
                shutil.rmtree(destination_path) 
                logging.info(f"Existing directory '{destination_path}' removed.")
            
            # Construct the git clone command
            command = ['git', 'clone', repo_url, destination_path]
            
            # Execute the command using subprocess.run for better error handling and output capture.
            result = subprocess.run(command, capture_output=True, text=True, check=True)
            
            logging.info(f"Cloning successful. Cloned to: {destination_path}")
            logging.debug(result.stdout)
            return True
        
        except subprocess.CalledProcessError as e:
            logging.error(f"Attempt {attempts + 1} failed with error: {e}")
            logging.debug(f"Error Output:\n{e.stderr}")
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")
        
        attempts += 1
        if attempts < patience:
            logging.info("Retrying...")
            sleep(2 ** attempts)
        
    logging.error("Max retries reached. Cloning failed.")
    return False


def run_code2prompt(repo_local_path, repo_context_tree_path, include="*.java"):
    os.makedirs(os.path.dirname(repo_context_tree_path), exist_ok=True)
    cmd = ["code2prompt", repo_local_path, "--include", include, "--output", repo_context_tree_path]
    logging.info(f"Running code2prompt: {' '.join(cmd)}")
    res = subprocess.run(cmd, capture_output=True, text=True, check=True)
    return True, res.stdout, res.stderr

def generate_design_smell_analysis(api_key, design_smells_file, context_file, repo_name, patience=5):
    """Generates design smell analysis using Gemini 1.5 Flash with retry logic."""
    
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(MODEL)
    
    design_smells_data = load_yaml(design_smells_file)
    context_data = load_context(context_file)
    
    prompt = f"""
    You are a software design expert tasked with identifying design smells in a given codebase.

    Here is a description of potential design smells:

    {yaml.dump(design_smells_data)}

    Here is the project context, including the source code:
    {context_data}

    Analyze the provided code and identify any design smells present.
    """
    
    attempts = 0
    while attempts < patience:
        try:
            response = model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    response_mime_type='application/json',
                    response_schema=list[DesignSmell]
                )
            )
            
            if response.candidates and response.candidates[0].content:
                try:
                    json_data = json.loads(response.text)
                    if isinstance(json_data, list):
                        validated_smells = [DesignSmell(**item) for item in json_data]
                        return validated_smells
                    else:
                        logging.error("Error: The model did not output a list. Output was:", response.text)
                except (json.JSONDecodeError, ValidationError) as e:
                    logging.error(f"Error parsing JSON or validating with Pydantic: {e}")
                    logging.debug("Raw response text:", response.text)
            else:
                logging.warning("No content or candidates found in the response.")
        
        except Exception as e:
            logging.error(f"Attempt {attempts + 1} failed with error: {e}")
            
        attempts += 1
        if attempts < patience:
            logging.info("Retrying...")
            sleep(2 ** attempts)
        
    logging.error("Max retries reached. Returning an empty list.")
    return []
     
def generate_refactoring_suggestions(api_key, design_smells_file, context_file, design_smells_found, patience=5):
    """Generates refactoring suggestions for design smells one by one using Gemini 1.5 Flash with retry logic."""
    
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(MODEL)
    
    design_smells_data = load_yaml(design_smells_file)
    context_data = load_context(context_file)
    
    all_code_changes: List[CodeBlock] = []
    smell_name = design_smells_found.Design_Smell
    
    prompt = f"""
You are a software refactoring expert tasked with identifying and suggesting code changes to remove the "{smell_name}" design smell from some specific files in a codebase.

Here is the design smell information:
{design_smells_found}

Here is the project codebase context:
{context_data}"""
    
    attempts = 0
    while attempts < patience:
        try:
            response = model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    response_mime_type='application/json',
                    response_schema=list[CodeBlock]
                )
            )
            
            try:
                response_json = json.loads(response.text)
                if isinstance(response_json, list):
                    for item in response_json:
                        try:
                            code_change = CodeBlock(**item)
                            all_code_changes.append(code_change)
                        except Exception as e:
                            logging.error(f"Error parsing CodeBlock object: {e}, item: {item}")
                else:
                    logging.warning("Response is not a list, attempting to parse as single CodeBlock")
                    try:
                        code_change = CodeBlock(**response_json)
                        all_code_changes.append(code_change)
                    except Exception as e:
                        logging.error(f"Error parsing single CodeBlock object: {e}, response: {response_json}")
                return all_code_changes
            except json.JSONDecodeError as e:
                logging.error(f"JSONDecodeError: {e}")
                logging.debug(f"Response text was: {response.text}")
            
        except Exception as e:
            logging.error(f"Attempt {attempts + 1} failed with error: {e}")
        
        attempts += 1
        if attempts < patience:
            logging.info("Retrying...")
            sleep(2 ** attempts)
        
    logging.error("Max retries reached. Returning collected code changes.")
    return all_code_changes

def updating_local_code(refactoring_results):
    # Iterate through all refactored code blocks and update the files
    for code_block in refactoring_results:
        file_path = code_block.File_Path
        updated_code = code_block.Updated_Code

        if not os.path.exists(file_path):
            logging.error(f"File {file_path} does not exist. Skipping update.")
            continue

        try:
            # Write the updated code to the file
            with open(file_path, 'w') as f:
                f.write(updated_code)

            logging.info(f"File {file_path} has been updated successfully!")

        except Exception as e:
            logging.error(f"Failed to update file {file_path}: {e}")

    # Update the Context
    print(f"{REPO_LOCAL_PATH}, {REPO_CONTEXT_TREE_PATH}")
    s, o, e = run_code2prompt(REPO_LOCAL_PATH, REPO_CONTEXT_TREE_PATH)

def commit_changes(REPO_LOCAL_PATH, REPO_OWNER, REPO_NAME, BASE_BRANCH, BRANCH_NAME, COMMIT_MESSAGE, PR_TITLE, PR_BODY):
    try:
        # Checkout the base branch
        # subprocess.run(["git", "checkout", BASE_BRANCH], check=True, cwd=REPO_LOCAL_PATH)
        # subprocess.run(["git", "pull"], check=True) # Pull latest changes from remote
        
        # Clear any stashed changes to avoid multiple commits showing up
        subprocess.run(["git", "stash", "clear"], check=True, cwd=REPO_LOCAL_PATH)
        logging.info("Cleared any stashed changes.")

        # Check if the branch exists locally.
        result = subprocess.run(["git", "branch", "--list", BRANCH_NAME], capture_output=True, text=True, cwd=REPO_LOCAL_PATH)
        branch_exists_locally = BRANCH_NAME in result.stdout

        # Create or checkout the branch
        if branch_exists_locally:
            subprocess.run(["git", "checkout", BRANCH_NAME], check=True, cwd=REPO_LOCAL_PATH)
            logging.info(f"Checked out existing branch: {BRANCH_NAME}")
        else:
            subprocess.run(["git", "checkout", "-b", BRANCH_NAME], check=True, cwd=REPO_LOCAL_PATH)
            logging.info(f"Created and checked out new branch: {BRANCH_NAME}")

        # Stage changes
        subprocess.run(["git", "add", "."], check=True, cwd=REPO_LOCAL_PATH)
        logging.info("Changes staged for commit.")

        # Check for changes (important for preventing empty commits)
        result = subprocess.run(
            ["git", "status", "--porcelain"],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, cwd=REPO_LOCAL_PATH
        )

        if not result.stdout.strip():
            logging.info("No changes to commit.")
            return

        # Commit changes (locally, no global config)
        subprocess.run(["git", "commit", "-m", COMMIT_MESSAGE,
                        "--author", "onepieceIsReal <onepiece.iiith@gmail.com>"],  # Set author for this commit ONLY
                    check=True, capture_output=True, text=True, cwd=REPO_LOCAL_PATH)
        
        logging.info(f"Changes committed with message: {COMMIT_MESSAGE}")
        
        # Push changes (using token in URL)
        remote_url = f"https://{GITHUB_TOKEN}@github.com/{REPO_OWNER}/{REPO_NAME}.git"
        subprocess.run(["git", "push", "-u", remote_url, BRANCH_NAME], check=True, capture_output=True, text=True, cwd=REPO_LOCAL_PATH)
        logging.info(f"Changes pushed to remote branch: {BRANCH_NAME}")

        # Create pull request
        g = Github(GITHUB_TOKEN)
        repo = g.get_repo(f"{REPO_OWNER}/{REPO_NAME}")
        pr = repo.create_pull(
            title=PR_TITLE,
            body=PR_BODY,
            head=BRANCH_NAME,
            base=BASE_BRANCH
        )

        logging.info(f"Pull request created successfully: {pr.html_url}")

    except subprocess.CalledProcessError as e:
        logging.error(f"Command failed: {e}")
        if e.stderr:
            logging.error(f"stderr: {e.stderr}")
        if e.stdout:
            logging.error(f"stdout: {e.stdout}")
    except Exception as e:
        logging.error(f"Error: {str(e)}")
        


        

def commit_info_generation(api_key, smell, patience=5):
    """Generates a commit message for the changes made to address a design smell."""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(MODEL)
    
    prompt = f"""
Write a commit message for the changes made to address this design smell:
{smell}
"""
    
    attempts = 0
    while attempts < patience:
        try:
            response = model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    response_mime_type='application/json',
                    response_schema=CommitInfo
                )
            )
            
            try:
                response_json = json.loads(response.text)
                commit_info = CommitInfo(**response_json)
                logging.info(f"Commit message generated: {commit_info}")
                return commit_info
            except (json.JSONDecodeError, ValidationError) as e:
                logging.error(f"Error parsing commit message: {e}")
                logging.debug(f"Response text was: {response.text}")
            
        except Exception as e:
            logging.error(f"Attempt {attempts + 1} failed with error: {e}")
        
        attempts += 1
        if attempts < patience:
            logging.info("Retrying...")
            sleep(2 ** attempts)
        
    logging.error("Max retries reached. Returning default commit message.")
    return f"Refactored code to address design smell: {smell.Design_Smell}"

def preprocess():
    # Remove the existing working directory
    if os.path.exists(WORKING_DIRECTORY_PATH):
        logging.warning(f"Removing existing working directory: {WORKING_DIRECTORY_PATH}")
        shutil.rmtree(WORKING_DIRECTORY_PATH)

    # Remove ContextTree directory
    if os.path.exists("ContextTree"):
        logging.warning(f"Removing existing ContextTree directory")
        shutil.rmtree("ContextTree")

    success = clone_repo(REPO_URL, REPO_NAME, WORKING_DIRECTORY_PATH)
    if success:
        print("Cloning completed.")
    else:
        print("Cloning failed.")

    s, o, e = run_code2prompt(REPO_LOCAL_PATH, REPO_CONTEXT_TREE_PATH)
    print("code2prompt:", o)

/home/yash/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
preprocess()

design_smells_found = generate_design_smell_analysis(API_KEY, KNOWLEDGE_BASE_DESIGN_SMELLS_PATH, REPO_CONTEXT_TREE_PATH, REPO_NAME)

if design_smells_found:
    print("Design smells found:", len(design_smells_found))
    print(json.dumps([smell.dict() for smell in design_smells_found], indent=2))
else:
    print("No design smells found or an error occurred.")

for smell in design_smells_found:
    preprocess()
    print(smell)
    refactoring_results = generate_refactoring_suggestions(API_KEY, KNOWLEDGE_BASE_DESIGN_SMELLS_PATH, REPO_CONTEXT_TREE_PATH, smell)

    if refactoring_results:
        print(json.dumps([change.dict() for change in refactoring_results], indent=2))
    else:
        print("No refactoring suggestions found or an error occurred.")

    updating_local_code(refactoring_results)
    COMMIT_INFO = commit_info_generation(API_KEY,smell)
    try:
        COMMIT_MESSAGE = COMMIT_INFO.commit_message
    except:
        COMMIT_MESSAGE = "Refactored code to address design smells"
    try:
        PR_BODY = COMMIT_INFO.pull_request_body
    except:
        PR_BODY = "This PR contains refactored code to address design smells."
    try:
        PR_TITLE = COMMIT_INFO.pull_request_title
    except:
        PR_TITLE = "Refactoring Updates"
    try:
        BRANCH_NAME = COMMIT_INFO.brach_name
    except:
        BRANCH_NAME = f"refactor-branch_{time.time()}"

    commit_changes(REPO_LOCAL_PATH, REPO_OWNER, REPO_NAME, BASE_BRANCH, BRANCH_NAME, COMMIT_MESSAGE, PR_TITLE, PR_BODY)

2025-02-11 18:44:22,388 - WARNING - Removing existing working directory: ./WorkingDirectory
2025-02-11 18:44:22,393 - WARNING - Removing existing ContextTree directory
2025-02-11 18:44:23,469 - INFO - Cloning successful. Cloned to: ./WorkingDirectory/RefactoringSampleRepo
2025-02-11 18:44:23,469 - DEBUG - 
2025-02-11 18:44:23,470 - INFO - Running code2prompt: code2prompt ./WorkingDirectory/RefactoringSampleRepo --include *.java --output ContextTree/RefactoringSampleRepo.txt


Cloning completed.
code2prompt: [✓] Copied to clipboard successfully.
[✓] Prompt written to file: ContextTree/RefactoringSampleRepo.txt



/tmp/ipykernel_8603/2166586876.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(json.dumps([smell.dict() for smell in design_smells_found], indent=2))
2025-02-11 18:44:27,265 - WARNING - Removing existing working directory: ./WorkingDirectory
2025-02-11 18:44:27,267 - WARNING - Removing existing ContextTree directory


Design smells found: 2
[
  {
    "Design_Smell": "Switch Statement",
    "Files_Paths": [
      "/home/yash/Desktop/Sem8/SE/AutomatedRefactoring/WorkingDirectory/RefactoringSampleRepo/TennisGame.java"
    ],
    "Reasoning": "The TennisGame1 class uses switch statements to determine the score based on different score combinations. This violates the Open/Closed Principle because adding a new score condition would require modifying the switch statement.  Polymorphism would be a more flexible and maintainable solution."
  },
  {
    "Design_Smell": "Conditional Complexity",
    "Files_Paths": [
      "/home/yash/Desktop/Sem8/SE/AutomatedRefactoring/WorkingDirectory/RefactoringSampleRepo/TennisGame.java"
    ],
    "Reasoning": "The getScore() method uses nested conditional statements (if-else and switch) to determine the game's score. This makes the code hard to read, understand, and maintain.  Refactoring into smaller, more focused methods and potentially using polymorphism could improve

2025-02-11 18:44:28,165 - INFO - Cloning successful. Cloned to: ./WorkingDirectory/RefactoringSampleRepo
2025-02-11 18:44:28,165 - DEBUG - 
2025-02-11 18:44:28,166 - INFO - Running code2prompt: code2prompt ./WorkingDirectory/RefactoringSampleRepo --include *.java --output ContextTree/RefactoringSampleRepo.txt


...

2025-02-11 18:44:39,204 - INFO - Commit message generated: brach_name='refactor/tennis-game-polymorphism' pull_request_title='Refactor: Improve TennisGame1 score calculation using polymorphism' pull_request_body='This pull request addresses a design smell related to the use of a switch statement in the `TennisGame1` class.  The switch statement was used to determine the score based on different score combinations. This violates the Open/Closed Principle, making it difficult to add new score conditions without modifying the existing switch statement. \n\nThis refactoring replaces the switch statement with polymorphism, creating a more flexible and maintainable solution.  Adding new score conditions now simply requires implementing a new class, rather than altering existing code. This improves extensibility and reduces the risk of introducing bugs when adding new features.  Tests have been included to ensure that the functionality remains the same after the refactoring.' commit_message='

Cloning completed.
code2prompt: [✓] Copied to clipboard successfully.
[✓] Prompt written to file: ContextTree/RefactoringSampleRepo.txt

Design_Smell='Conditional Complexity' Files_Paths=['/home/yash/Desktop/Sem8/SE/AutomatedRefactoring/WorkingDirectory/RefactoringSampleRepo/TennisGame.java'] Reasoning="The getScore() method uses nested conditional statements (if-else and switch) to determine the game's score. This makes the code hard to read, understand, and maintain.  Refactoring into smaller, more focused methods and potentially using polymorphism could improve readability and maintainability."


/tmp/ipykernel_8603/2166586876.py:17: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(json.dumps([change.dict() for change in refactoring_results], indent=2))
2025-02-11 18:44:49,525 - INFO - File /home/yash/Desktop/Sem8/SE/AutomatedRefactoring/WorkingDirectory/RefactoringSampleRepo/TennisGame.java has been updated successfully!
2025-02-11 18:44:49,526 - INFO - Running code2prompt: code2prompt ./WorkingDirectory/RefactoringSampleRepo --include *.java --output ContextTree/RefactoringSampleRepo.txt


[
  {
    "File_Path": "/home/yash/Desktop/Sem8/SE/AutomatedRefactoring/WorkingDirectory/RefactoringSampleRepo/TennisGame.java",
    "Updated_Code": "public class TennisGame1 implements TennisGame {\n    private int m_score1 = 0;\n    private int m_score2 = 0;\n    private String player1Name;\n    private String player2Name;\n\n    public TennisGame1(String player1Name, String player2Name) {\n        this.player1Name = player1Name;\n        this.player2Name = player2Name;\n    }\n\n    public void wonPoint(String playerName) {\n        if (playerName == \"player1\")\n            m_score1 += 1;\n        else\n            m_score2 += 1;\n    }\n\n    public String getScore() {\n        if (m_score1 == m_score2) {\n            return getEqualScore();\n        } else if (m_score1 >= 4 || m_score2 >= 4) {\n            return getHighScore();\n        } else {\n            return getNormalScore();\n        }\n    }\n\n    private String getEqualScore() {\n        switch (m_score1) {\n        

2025-02-11 18:44:52,465 - INFO - Commit message generated: brach_name='refactor/reduce-conditional-complexity' pull_request_title='Refactor: Improve Readability of TennisGame.getScore()' pull_request_body="This commit refactors the `getScore()` method in `TennisGame.java` to address excessive conditional complexity. The original method contained nested if-else and switch statements, making it difficult to read and maintain. This refactoring improves readability and maintainability by:\n\n- Extracting smaller, more focused methods to handle specific scoring scenarios.\n- Improving overall code structure and reducing nesting. \n\nThis change enhances the code's clarity and makes future modifications easier." commit_message='Refactor: Reduce conditional complexity in getScore() method'
2025-02-11 18:44:52,468 - INFO - Cleared any stashed changes.
Switched to a new branch 'refactor/reduce-conditional-complexity'
2025-02-11 18:44:52,475 - INFO - Created and checked out new branch: refactor/